<a href="https://colab.research.google.com/github/gmprovan/CS4705/blob/main/Assignment_2_Learning_Bayesian_Networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The purpose of this notebook is to gain some practice in learning graphical models. Your goal is to:

1.   load the Breast Cancer (categorical) data set: https://archive.ics.uci.edu/ml/datasets/breast+cancer
2.   keep the last 20% of the data for testing
3.   compare the performance of 3 learned models on the test data: naive Bayes, tree-structured BN (using the Chow-Liu algorithm), and BN



In [1]:
import pandas as pd
import numpy as np
from pgmpy.estimators import K2Score, BicScore
from pgmpy.models import BayesianNetwork

c:\Users\anido\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Load data
data = pd.read_csv('./data/breast-cancer.data',
                   names=['Class', 'age', 'menopause', 'tumor-size', 'inv-nodes', 'node-caps', 'deg-malig', 'breast', 'breast-quad', 'irradiat'])

In [3]:
# Change ? to NaN
data[data == "?"] = np.nan

In [4]:
data = data[data["age"] != "20-29"]
data = data[data["inv-nodes"] != "24-25"]

# Train models

In [5]:
from sklearn.model_selection import train_test_split

# separate train and test data
dtrain, dtest = train_test_split(data.fillna(data.mode().iloc[0]), test_size=0.2, random_state=1)

## Naive Bayes model:

For the naive bayes I use the maximum likelihood estimator

In [21]:
model_nb = BayesianNetwork(model_nb.edges())

In [22]:
from pgmpy.models.NaiveBayes import NaiveBayes
from pgmpy.estimators import MaximumLikelihoodEstimator
# create the structure manually to create model_struct

#model_nb = NaiveBayes(data.columns[1:], data.columns[0])

model_nb.fit(dtrain, estimator=MaximumLikelihoodEstimator)

print(model_nb.get_cpds('Class'))

# print the score
print("test:\n", "K2:", K2Score(dtest).score(model_nb), "| Bic:", BicScore(dtest).score(model_nb))
print("train:\n", "K2:", K2Score(dtrain).score(model_nb), "| Bic:", BicScore(dtrain).score(model_nb))

+-----------------------------+----------+
| Class(no-recurrence-events) | 0.688596 |
+-----------------------------+----------+
| Class(recurrence-events)    | 0.311404 |
+-----------------------------+----------+
test:
 K2: -613.0600117480079 | Bic: -655.5336891240778
train:
 K2: -2331.4476306110555 | Bic: -2380.6982978994574


In [41]:
(model_nb.predict(dtest[data.columns[1:]]).values == dtest[["Class"]].values).sum()/len(dtest)

100%|██████████| 57/57 [00:00<00:00, 1527.25it/s]


0.7017543859649122

The scores obtained are good, we do seem to have a big difference between test and training data, which means that this model is not a good representation for general data.

I decide to check the predictive accuracy independently, because I do not quite understand the previous scores.
Since I get an error trying to use the .predict for naive bayes, I have implemented the predictions generation for this specific case.

In [31]:
predictions = []

for row in dtest.values:
    result = [1, 1]
    # test an example
    for j, d in enumerate(model_nb.cpds[0].values):
        for i, val in enumerate(row[1:]):
            if model_nb.get_cpds(data.columns[i+1]).name_to_no[data.columns[i+1]].get(val, None) != None:
                result[j] *= float(model_nb.get_cpds(data.columns[i+1]).values[model_nb.get_cpds(data.columns[i+1]).name_to_no[data.columns[i+1]][val]][j])
            else: 
                result[j] *= 0.000001

    predictions += [model_nb.cpds[0].state_names["Class"][result.index(max(result))]]
print("Test accuracy: ", (predictions == dtest.Class.values).sum()/len(predictions))

Test accuracy:  0.6140350877192983


In [24]:
predictions = []

for row in dtest.values:
    result = [1, 1]
    # test an example
    for j, d in enumerate(model_nb.cpds[0].values):
        for i, val in enumerate(row[1:]):
            if model_nb.get_cpds(data.columns[i+1]).name_to_no[data.columns[i+1]].get(val, None) != None:
                result[j] *= float(model_nb.get_cpds(data.columns[i+1]).values[model_nb.get_cpds(data.columns[i+1]).name_to_no[data.columns[i+1]][val]][j])
            else: 
                result[j] *= 0.000001

    predictions += [model_nb.cpds[0].state_names["Class"][result.index(max(result))]]
print("Test accuracy: ", (predictions == dtest.Class.values).sum()/len(predictions))

predictions = []

for row in dtrain.values:
    result = [1, 1]
    # test an example
    for j, d in enumerate(model_nb.cpds[0].values):
        for i, val in enumerate(row[1:]):
            if model_nb.get_cpds(data.columns[i+1]).name_to_no[data.columns[i+1]].get(val, None) != None:
                result[j] *= float(model_nb.get_cpds(data.columns[i+1]).values[model_nb.get_cpds(data.columns[i+1]).name_to_no[data.columns[i+1]][val]][j])
            else: 
                result[j] *= 0.000001

    predictions += [model_nb.cpds[0].state_names["Class"][result.index(max(result))]]
print("Train accuracy: ", (predictions == dtrain.Class.values).sum()/len(predictions))

Test accuracy:  0.6140350877192983
Train accuracy:  0.75


Observing these results, the model seems to work with somewhat acceptable performance, we do see that there is a `10%` difference between train and test accuracy, which is consistent with the previous scores. We can probably get a better model using more complex networks, so I expect the following models to perform better than this one.

## Tree-structured model:

In [8]:
from pgmpy.estimators import TreeSearch

# Impute missing data with the mode
dtrain = dtrain.fillna(dtrain.mode().iloc[0])

# learn graph structure using TreeSearch and Chow-Liu algorithm
est = TreeSearch(dtrain, root_node="Class")
dag = est.estimate(estimator_type="chow-liu")

from pgmpy.estimators import BayesianEstimator

# there are many choices of parametrization, here is one example
model = BayesianNetwork(dag.edges())
model.fit(
    data, estimator=BayesianEstimator, prior_type="dirichlet", pseudo_counts=0.1
)


Building tree: 100%|██████████| 45/45.0 [00:00<00:00, 1260.25it/s]

In [9]:
# print the score
print("test:\n", "K2:", K2Score(dtest).score(model), "| Bic:", BicScore(dtest).score(model))
print("train:\n", "K2:", K2Score(dtrain).score(model), "| Bic:", BicScore(dtrain).score(model))

test:
 K2: -602.3680239908574 | Bic: -800.6569955287398
train:
 K2: -2236.4249916106974 | Bic: -2546.7317696896125


In [10]:
# print the accuracy
print("test: ", (model.predict(dtest[data.columns[1:]]).values == dtest[[data.columns[0]]].reset_index(drop=True).values).sum() / len(dtest))
print("train: ", (model.predict(dtrain[data.columns[1:]]).values == dtrain[[data.columns[0]]].reset_index(drop=True).values).sum() / len(dtrain))

100%|██████████| 57/57 [00:00<00:00, 1047.43it/s]

test:  0.7543859649122807


100%|██████████| 215/215 [00:00<00:00, 925.53it/s]


train:  0.7105263157894737


Comparing the scores of the two models, we can see that the Naive Bayes model has a lower score for all the values. This model seems to be good, specially since we get a very similar accuracy for the train and test set, which seems to indicate that the model is a good generalization and does not overfit the training data.

Additionallly we can see the predictive accuracy for the other variables, since it is a Bayesian Model, it is easy to predict the value for any missing variable from all the others using the same model.

In [11]:
preds = {}
for col in data:
    preds[col] = [((model.predict(dtest.drop(columns=[col])).values == dtest[[col]].reset_index(drop=True).values).sum() / len(dtest)),
                  ((model.predict(dtrain.drop(columns=[col])).values == dtrain[[col]].reset_index(drop=True).values).sum() / len(dtrain))]

100%|██████████| 216/216 [00:00<00:00, 498.92it/s]


In [12]:
print("test", "|", "train")
for col in preds:
    print(col, preds[col])

test | train
Class [0.7543859649122807, 0.7105263157894737]
age [0.5263157894736842, 0.5131578947368421]
menopause [0.8947368421052632, 0.8114035087719298]
tumor-size [0.24561403508771928, 0.2850877192982456]
inv-nodes [0.8070175438596491, 0.8070175438596491]
node-caps [0.8421052631578947, 0.8859649122807017]
deg-malig [0.543859649122807, 0.5131578947368421]
breast [0.6491228070175439, 0.6754385964912281]
breast-quad [0.45614035087719296, 0.5263157894736842]
irradiat [0.8245614035087719, 0.7850877192982456]


We see some variables that are very bad in this case, like `tumor-size`, `deg-malign`, `breast-quad` and `age`, whereas others are even better than the `Class` variable, it is very difficult to obtain a good score for all the variables but in the same way it is good to know we have some way to predict other variables if we have any problems.

This can also help us understand which variables are actually important to the model, we could continue trying some different subset of the variables to try to obtain a better model.

The following approach takes this into account and in the structure building it may be possible that some variables are not used.

## Bayesian Network model:

### Learn Structure:

For the model creation I use hill climbing, because the whole searching space is very big this approach will get to a local minimum, if I had more time I might have tried initializing with different random structures or include some kind of randomness in the search to try to get to a better solution.

In [13]:
from pgmpy.estimators import HillClimbSearch, BicScore, K2Score, BDeuScore

est = HillClimbSearch(dtrain)

# estimate the DAG
model = BayesianNetwork(est.estimate(scoring_method=BicScore(dtrain), show_progress=False).edges()) 
print(model.edges())

model.fit(dtrain, estimator=BayesianEstimator, prior_type="dirichlet", pseudo_counts=0.1)

# print the score
print("test:\n", "K2:", K2Score(dtest).score(model), "| Bic:", BicScore(dtest).score(model))
print("train:\n", "K2:", K2Score(dtrain).score(model), "| Bic:", BicScore(dtrain).score(model))

#print((model.predict(dtest[list(model.nodes()._nodes.keys())].drop(columns=["Class"])).values == dtest[[data.columns[0]]].reset_index(drop=True).values).sum() / len(dtest))
#print((model.predict(dtrain[list(model.nodes()._nodes.keys())].drop(columns=["Class"])).values == dtrain[[data.columns[0]]].reset_index(drop=True).values).sum() / len(dtrain))


[('menopause', 'age'), ('node-caps', 'inv-nodes'), ('node-caps', 'deg-malig'), ('deg-malig', 'Class'), ('breast', 'breast-quad'), ('irradiat', 'node-caps')]
test:
 K2: -455.10833332994423 | Bic: -479.57666873007065
train:
 K2: -1701.8622890331535 | Bic: -1729.0220587763633


These results are the worst among all the models, this may be because the model gets stuck in a local minimum and stops. That is why I would try to initialize the model in a different way to try to avoid getting stuck in this place.

The same as before I have not been able to use the prediction tools in the library so I created one that works for this model so I can compare the accuracy for each model.

He score and predict function

In [14]:
dtrain.head()

,Class,age,menopause,tumor-size,inv-nodes,node-caps,deg-malig,breast,breast-quad,irradiat
270,recurrence-events,50-59,ge40,30-34,6-8,yes,3,left,right_low,no
38,no-recurrence-events,40-49,premeno,10-14,0-2,no,2,left,left_low,no
160,no-recurrence-events,40-49,premeno,25-29,0-2,no,3,right,left_up,yes
285,recurrence-events,50-59,ge40,30-34,3-5,no,3,left,left_low,no
171,no-recurrence-events,30-39,premeno,15-19,0-2,no,1,left,left_low,no


In [16]:
predictions = []

for row in dtest.values:
    result = [1, 1]
    # test an example
    for j, d in enumerate(model.cpds[0].values):
        if model.get_cpds("deg-malig").name_to_no["deg-malig"].get(row[6], None) != None:
            result[j] *= float(model.get_cpds("deg-malig").values[model.get_cpds("deg-malig").name_to_no["deg-malig"][row[6]]][j])
        else: 
            result[j] *= 0.000001

    predictions += [model.cpds[0].state_names["Class"][result.index(max(result))]]
print("Test accuracy: ", (predictions == dtest.Class.values).sum()/len(predictions))

predictions = []

for row in dtrain.values:
    result = [1, 1]
    # test an example
    for j, d in enumerate(model.cpds[0].values):
        if model.get_cpds("deg-malig").name_to_no["deg-malig"].get(row[6], None) != None:
            result[j] *= float(model.get_cpds("deg-malig").values[model.get_cpds("deg-malig").name_to_no["deg-malig"][row[6]]][j])
        else: 
            result[j] *= 0.000001

    predictions += [model.cpds[0].state_names["Class"][result.index(max(result))]]
print("Train accuracy: ", (predictions == dtrain.Class.values).sum()/len(predictions))

IndexError: list index out of range

These results are very similar to those obtained with the Naive Bayes, which is very disapointing. Even thought we obtain these results, I do think this last approach is more capable.

## Conclusions

**Naive Bayes:**

|      |         K2         |        Bic        |      Accuracy      |
|------|--------------------|-------------------|--------------------|
| test | -639.5689356351188 | -683.769991489145 | 0.6379310344827587 |
| train| -2327.1730910369565|-2377.1741867518376| 0.7368421052631579 |


**Tree Structured Bayesian Classifier:**

|      |         K2         |        Bic        |      Accuracy      |
|------|--------------------|-------------------|--------------------|
| test | -622.3407496923875 | -849.9164833843682| 0.7241379310344828 |
| train| -2233.9267785881393|-2544.746122943686 | 0.7280701754385965 |


**Bayesian Network:**

|      |         K2         |        Bic        |      Accuracy      |
|------|--------------------|-------------------|--------------------|
| test | -479.60984292221355|-502.21788576729847| 0.6551724137931034 |
| train| -1696.2692882744664|-1722.0429324104523| 0.7412280701754386 |


From these results we can say that the model that performs the best is 
From these results it is interesting to mark that even though we get a better score for the BN accuracy but the other scores are significantly higher.

----

Finally learn a Bayesian network. **First learn the structure, and then the parameters.**

# **Learning Bayesian Networks**

We now want to learn a Bayesian network, given a set of sample data. Learning a Bayesian network can be split into two problems:

 **Structure learning**: Given a set of data samples, estimate a DAG that captures the dependencies between the variables.

  **Parameter learning**: Given a set of data samples and a DAG that captures the dependencies between the variables, estimate the (conditional) probability distributions of the individual variables.


Methods for doing this include:

Structure learning for discrete, fully observed networks:
    
*    Score-based structure estimation (BIC/BDeu/K2 score; exhaustive search, hill climb/tabu search)
*   Constraint-based structure estimation (PC)

Parameter learning for discrete nodes:

*   Maximum Likelihood Estimation
*   Bayesian Estimation
    





**Structure Learning**

You can use MLE or Bayesian estimation methods.

MLE State counts

To make sense of the given data, we can start by counting how often each state of the variable occurs. If the variable is dependent on parents, the counts are done conditionally on the parents states, i.e. for separately for each parent configuration:

**Bayesian Parameter Estimation**


The Bayesian Parameter Estimator starts with already existing prior CPDs, that express our beliefs about the variables before the data was observed. Those "priors" are then updated, using the state counts from the observed data. 

One can think of the priors as consisting in pseudo state counts, that are added to the actual counts before normalization. Unless one wants to encode specific beliefs about the distributions of the variables, one commonly chooses uniform priors, i.e. ones that deem all states equiprobable.

A very simple prior is the so-called K2 prior, which simply adds 1 to the count of every single state. A somewhat more sensible choice of prior is BDeu (Bayesian Dirichlet equivalent uniform prior). For BDeu we need to specify an equivalent sample size N and then the pseudo-counts are the equivalent of having observed N uniform samples of each variable (and each parent configuration). 

**Maximum Likelihood Estimation**


A natural estimate for the CPDs is to simply use the relative frequencies, with which the variable states have occured. 

This approach is Maximum Likelihood Estimation (MLE). According to MLE, we should fill the CPDs in such a way, that $P(\text{data}|\text{model})$ is maximal. This is achieved when using the relative frequencies.  pgmpy supports MLE as follows:


mle.estimate_cpd(variable) computes the state counts and divides each cell by the (conditional) sample size. The mle.get_parameters()-method returns a list of CPDs for all variable of the model.

The built-in fit()-method of BayesianModel provides more convenient access to parameter estimators:

# **Structure Learning**




To learn model structure (a DAG) from a data set, there are two broad techniques:

*   score-based structure learning
*   constraint-based structure learning

In this assignment focus on the score-based approach.


# **Score-based Structure Learning**


This approach construes model selection as an optimization task. It has two building blocks:

A scoring function $s_D\colon M \to \mathbb R$ that maps models to a numerical score, based on how well they fit to a given data set $D$.
A search strategy to traverse the search space of possible models $M$ and select a model with optimal score.


**Scoring functions**


Commonly used scores to measure the fit between model and data are Bayesian Dirichlet scores such as BDeu or K2 and the Bayesian Information Criterion (BIC, also called MDL). 


**Search strategies**


The search space of DAGs is super-exponential in the number of variables and the above scoring functions allow for local maxima. The first property makes exhaustive search intractable for all but very small networks, the second prohibits efficient local optimization algorithms to always find the optimal structure. Thus, identifiying the ideal structure is often not tractable. Despite these bad news, heuristic search strategies often yields good results.


Heuristic search: HillClimbSearch implements a greedy local search that starts from the DAG start (default: disconnected DAG) and proceeds by iteratively performing single-edge manipulations that maximally increase the score. The search terminates once a local maximum is found.


The estimated values in the CPDs are now more conservative. 

BayesianEstimator, too, can be used via the fit()-method. 




# **Discussion**

Please critically compare the performance of the 3 different models.